# ANÁLISIS MULTIANUAL DE MONTO EJECUTADO POR OBJETO DE GASTO Y MES (CON FECHA DE CORTE MÁXIMA)

Esta sección realiza un análisis exhaustivo del presupuesto ejecutado para todos los objetos de gasto, considerando todos los años disponibles en el dataset (`df_all_years`). El objetivo es obtener, para cada combinación de año, objeto de gasto y mes, el monto ejecutado correspondiente a la fecha de corte más reciente publicada para ese grupo. 

**Metodología:**
- Se filtran los registros con `montoEjecutado > 0`.
- Para cada año, objeto de gasto y mes, se identifica la fechaCorte máxima (última actualización disponible).
- Se seleccionan todos los registros correspondientes a esa fecha de corte máxima.
- Se suman los montos ejecutados por año, objeto de gasto y mes, generando un reporte detallado y un resumen anual por objeto de gasto.
- Se calcula el total general ejecutado para todos los años y objetos de gasto.

**Utilidad:**
- Permite comparar la ejecución presupuestaria real consolidada por objeto de gasto y año, utilizando siempre la información más actualizada disponible para cada mes.
- Es especialmente útil para auditoría, control de gestión y conciliación con cifras oficiales publicadas.
- El reporte final facilita la visualización de la evolución y distribución del gasto público a lo largo de los años y entre diferentes conceptos.

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

csv_path = os.getenv("csv_path")
# Update the filename if needed

# Read the CSV file into a DataFrame
df_all_years = pd.read_csv(csv_path, encoding="utf-8", sep=",")

In [ ]:
# Filtrar el DataFrame para montoEjecutado > 0 (sin filtrar por año ni objetoGasto)
df_multi_filtrado = df_all_years[df_all_years['montoEjecutado'] > 0].copy()
df_multi_filtrado.sort_values(["anio", "objetoGasto", "mes"], ascending=[True, True, False], inplace=True)
df_multi_filtrado.head()

### Paso 1: Obtener la fechaCorte máxima por año, objetoGasto y mes

In [ ]:
# Para cada año, objetoGasto y mes, obtener la fechaCorte máxima
df_fechas_max_multi = df_multi_filtrado.groupby(['anio', 'objetoGasto', 'mes'], as_index=False)['fechaCorte'].max()
df_fechas_max_multi.head()

### Paso 2: Filtrar para quedarnos solo con los registros de la fechaCorte máxima de cada año, objetoGasto y mes

In [ ]:
# Unir para obtener todos los registros de la fechaCorte máxima de cada año, objetoGasto y mes
df_max_fecha_multi = pd.merge(df_multi_filtrado, df_fechas_max_multi, on=['anio', 'objetoGasto', 'mes', 'fechaCorte'])
df_max_fecha_multi = df_max_fecha_multi[['anio','objetoGasto','mes','montoEjecutado','montoTransferido','montoPagado','mesCorte','anioCorte','fechaCorte']]
df_max_fecha_multi = df_max_fecha_multi.sort_values(['anio','objetoGasto','fechaCorte','mes'], ascending=[True,True,False,False])
df_max_fecha_multi.head()

### Paso 3: Sumar montoEjecutado por año, objetoGasto y mes, y mostrar el total general

In [ ]:
# Sumar montoEjecutado por año, objetoGasto y mes
df_suma_mensual_multi = df_max_fecha_multi.groupby(['anio','objetoGasto','mes'], as_index=False)['montoEjecutado'].sum().sort_values(['anio','objetoGasto','mes'], ascending=[True,True,False])
df_suma_mensual_multi

In [ ]:
# Sumar el total general
total_general_multi = df_max_fecha_multi['montoEjecutado'].sum()
print(f"TOTAL GENERAL montoEjecutado (todos los años): {total_general_multi:,.2f}")

### Paso 4: Reporte resumido por anio y objetoGasto (suma de montoEjecutado)

En este paso, se genera un reporte que agrupa los datos por año y objeto de gasto, mostrando la suma total de montoEjecutado para cada combinacón. Esto permite visualizar la ejecución presupuestaria consolidada por concepto y año, facilitando el análisis comparativo y la conciliaciónn con cifras oficiales.

In [ ]:
# Agrupar por anio y objetoGasto, sumando montoEjecutado
reporte_resumido = df_max_fecha_multi.groupby(['anio', 'objetoGasto'], as_index=False)['montoEjecutado'].sum()
reporte_resumido = reporte_resumido.sort_values(['anio', 'objetoGasto'])
reporte_resumido

### Paso 5: Reporte anual consolidado (sin agrupar por objetoGasto)

En este paso, se genera un reporte que agrupa únicamente por año, sumando el montoEjecutado total de todos los objetos de gasto para cada año. Esto permite visualizar la ejecución presupuestaria consolidada anual, facilitando la comparación global entre años y la conciliación con cifras oficiales.

In [ ]:
# Agrupar solo por anio, sumando montoEjecutado de todos los objetos de gasto
reporte_anual = df_max_fecha_multi.groupby(['anio'], as_index=False)['montoEjecutado'].sum()
reporte_anual = reporte_anual.sort_values(['anio'])
reporte_anual

In [ ]:
# Mostrar el total general de montoEjecutado para el reporte anual consolidado
suma_total_anual = reporte_anual['montoEjecutado'].sum()
print(f"TOTAL GENERAL montoEjecutado (reporte anual consolidado): {suma_total_anual:,.2f}")

### Paso 5: Reporte resumido por año (suma de montoEjecutado)

En este paso, se genera un reporte que agrupa los datos únicamente por año, mostrando la suma total de montoEjecutado para cada año. Esto permite visualizar la ejecución presupuestaria anual consolidada, facilitando el análisis global y la comparación interanual.

In [ ]:
# Agrupar solo por anio, sumando montoEjecutado
reporte_anual = df_max_fecha_multi.groupby(['anio'], as_index=False)['montoEjecutado'].sum()
reporte_anual = reporte_anual.sort_values(['anio'])
reporte_anual

# Mostrar el total general de montoEjecutado para el reporte anual
suma_total_anual = reporte_anual['montoEjecutado'].sum()
print(f"TOTAL GENERAL montoEjecutado (reporte anual): {suma_total_anual:,.2f}")